In [ ]:
# import libraries


### Milestone 1: Getting to know your data, due Wednesday, April 5, 2017

In the beginning you should get acquainted with the data sources and do some EDA. Sign up for the TMDb [API](https://www.themoviedb.org/documentation/api), and try to download the poster of your favorite movie from within your notebook. Compare the genre entries of IMDb and TMDb for this movie and see if they are the same. Think about and write down some questions that you would like to answer in the following weeks. Keep the storytelling aspect of your final report in mind and do some pen and paper sketches about the visualizations you would like to produce. Include photographs of those sketches in your notebook. 

Most of the time a data scientist spends on a project is spend on cleaning the data. We are lucky that the data we have is already pretty clean. The Python interface to the IMDb ftp files does a lot of the additional work of cleaning as well. However, you will notice that the genre list for each movie from both databases can have different lengths. This needs to be changed in order to train a model to predict the movie genre. It is up to you to think about possible ways to address this problem and to implement one of them. There is no absolute right answer here. It depends on your interests and which questions you have in mind for the project. 

Optionally, you could also scrape additional data sources, such as Wikipedia, to obtain plot summaries. That data may give you additional useful features for genera classification. 

To guide your decision process, provide at least one visualization of how often genres are mentioned together in pairs. Your visualization should clearly show if a horror romance is more likely to occur in the data than a drama romance.

The notebook to submit for this milestone needs to at least include:

- API code to access the genre and movie poster path of your favorite movie
- Genre for this movie listed by TMDb and IMDb
- A list of the 10 most popular movies of 2016 from TMDb and their genre obtained via the API
- Comments on what challenges you see for predicting movie genre based on the data you have, and how to address them 
- Code to generate the movie genre pairs and a suitable visualization of the result
- Additional visualization sketches and EDA with a focus on movie genres
- A list of questions you could answer with this and related data. Get creative here!

The EDA questions do not necessarily have to tie into the modeling part later on. Think freely about things that might be interesting, like which actors are very specific to a genre? Are action movies more prone to producing sequels than romances? However, as you keep the focus on movie genres, think also about correlations you might discover that can help building features from the metadata for prediction. Is the length of a movie title correlated with genre?


### Set up TMDb API

In [10]:
import tmdbsimple as tmdb
tmdb.API_KEY = ''

In [6]:
search = tmdb.Search()
response = search.movie(query='Good Will Hunting')

In [9]:
response['results'][0]

{u'adult': False,
 u'backdrop_path': u'/8vA6dxBzRqSajvqdmTLtoguAa3T.jpg',
 u'genre_ids': [18],
 u'id': 489,
 u'original_language': u'en',
 u'original_title': u'Good Will Hunting',
 u'overview': u'Will Hunting, a janitor at MIT, has a gift for mathematics but needs help from a psychologist to find direction in his life.',
 u'popularity': 3.903482,
 u'poster_path': u'/jq8LjngZ7XZEQge5JFTdOGMrHyZ.jpg',
 u'release_date': u'1997-12-05',
 u'title': u'Good Will Hunting',
 u'video': False,
 u'vote_average': 7.8,
 u'vote_count': 2045}

In [26]:
import requests

In [38]:
title = 'Finding Nemo'
title = title.replace(' ', '%20')

In [39]:
r = requests.get('https://api.themoviedb.org/3/search/movie?api_key=9767d17413ec9d9729c2cca238df02da&language=en-US&query='+ title +'&page=1&include_adult=false')

In [40]:
results = r.json()[u'results'][0]

In [41]:
results

{u'adult': False,
 u'backdrop_path': u'/n2vIGWw4ezslXjlP0VNxkp9wqwU.jpg',
 u'genre_ids': [16, 10751],
 u'id': 12,
 u'original_language': u'en',
 u'original_title': u'Finding Nemo',
 u'overview': u"A tale which follows the comedic and eventful journeys of two fish, the fretful Marlin and his young son Nemo, who are separated from each other in the Great Barrier Reef when Nemo is unexpectedly taken from his home and thrust into a fish tank in a dentist's office overlooking Sydney Harbor. Buoyed by the companionship of a friendly but forgetful fish named Dory, the overly cautious Marlin embarks on a dangerous trek and finds himself the unlikely hero of an epic journey to rescue his son.",
 u'popularity': 7.110192,
 u'poster_path': u'/zjqInUwldOBa0q07fOyohYCWxWX.jpg',
 u'release_date': u'2003-05-30',
 u'title': u'Finding Nemo',
 u'video': False,
 u'vote_average': 7.5,
 u'vote_count': 4278}

In [ ]:
r = requests.get('https://api.themoviedb.org/3/search/movie?api_key=9767d17413ec9d9729c2cca238df02da&language=en-US&query='+ title +'&page=1&include_adult=false')